
## Iterable, Iterator and Generator

### Iterable and Iterator

Iteration -> Repetition of a process.

__Iterable__ is a type of object which would generate an Iterator when passed to in-built method `iter()`.

__Iterator__ is an object which is used to iterate over an iterable object using `next()` method, which returns the next item of the iterable object. Any object that has a `next()` method is therefore an iterator.

__NOTE__: List, Tuple, Set, Frozenset, Dictionary are in-built iterable objects. They are iterable containers from which you can get an iterator.

This is what happens.

![](../img/iterable-vs-iterator.png)

In [ ]:
## Let's see an example
my_tuple = ["apple", "banana", "cherry"]
iterated_tuple = iter(my_tuple)
print(type(iterated_tuple))
print(next(iterated_tuple))
print(next(iterated_tuple))
print(next(iterated_tuple))

In [ ]:
## same thing can be written using for loop
my_tuple = ("apple", "banana", "cherry")
for i in my_tuple:
    print(i)

__How for loop actually works?__

The for loop can iterate over any iterable.

```python
for element in iterable:
    # do something with element
```
is actually implemented as

```python
# create an iterator object from that iterable
iter_obj = iter(iterable)# infinite loop
while True:
    try:
        # get the next item
        element = next(iter_obj)
        # do something with element
    except StopIteration:
        # if StopIteration is raised, break from loop
        break
```

* The for loop creates an iterator object internally, `iter_obj` by calling `iter()` on the iterable.
* Inside the while loop, it calls `next()` to get the next element and executes further
* After all the items exhaust, __`StopIteration`__ exception is raised which is internally caught and the loop ends.

To get a better sense of the internals of an iterator, let’s build an iterator producing the Fibonacci numbers.

In [ ]:
from itertools import islice

class fib:
   def __init__(self):
      self.prev = 0
      self.curr = 1

   def __iter__(self):
      return self

   def __next__(self):
      value = self.curr
      self.curr += self.prev
      self.prev = value
      return value
        
f = fib()
print(list(islice(f, 0, 10)))

## Generator

A lot of overhead in building an iterator:

* implement a class with `iter()` and `next()` methods
* raise StopIteration when there was no values to be returned
* makes the code lengthy

Python Generators are a simple way of creating iterators. All the above mentioned overhead are automatically handled by generators in Python.

__Generator__ is a block of code, same as defining a function, having a __yield__ statement instead of a __return__ statement. If a function contains __at least one yield statement__ (it may contain other yield or return statements), it becomes a generator!

The yield statement suspends function’s execution and sends a value back to caller, but retains enough capability to enable function to resume where it is left off. When resumed, the function continues execution immediately after the last yield run. This allows its code to produce a series of values over time, rather than computing them at once and sending them back like a list. __We should use yield when we want to iterate over a sequence, but don’t want to store the entire sequence in memory__.

This is how the above Fibonacci number code looks like using generator.

In [ ]:
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr
        
f = fib()
res = list(islice(f, 0, 10))
print(res)

Let’s see what happened.

Take note that __fib__ is defined as a normal Python function, not as class. However, there’s __no `return` keyword__ inside the function body. The `return` value of the function will be a generator.

* Now when `f = fib()` is called, the generator is instantiated and returned. No code will be executed at this point.To be explicit: the line `prev, curr = 0, 1` is not executed yet.
* Then, this generator instance `f` is wrapped in an `islice()`. This is itself also an __iterator__. Again, no code executed.
* Now, this iterator is wrapped in a `list()`, which will take the argument `islice(<generator-instance>)` and build a list from it. To do so, it will start calling `next()` on the `islice()` instance, which in turn will start calling `next()` on our `f` instance.
* On the first call, the code `prev, curr = 0, 1` gets executed, then the `while True` loop is entered, and then it encounters the `yield curr` statement. It will produce the value that’s currently in the `curr` variable and become idle again. This value is passed to the `islice()` wrapper, which will produce the value (1 in this case) and `list` will add the value to the variable `res`.
* Then, `list` asks `islice()` for the next value, which will ask `f` for the next value, which will unpause `f` from its previous state, resuming with the statement `prev, curr = curr, prev + curr`.
* Then it re-enters the next iteration of the `while True` loop, and hits the `yield curr` statement, returning the next value of `curr`.
* This happens until the output list is 10 elements long. When `list()` asks `islice()` for the 11th value, `islice()` will raise a __`StopIteration`__ exception, indicating that the end has been reached, and list will return the result: a list of 10 items, containing the first 10 Fibonacci numbers.

There are two types of generators in Python: __generator functions__ and __generator expressions__. 
A generator function is any function in which the keyword `yield` appears in its body. We just saw an example of that. 
Generator expression is equivalent to list comprehension.

To avoid any confusion between iterable, iterator, generator, generator expression, a `list`, `set`, `dict` comprehension, check out this diagram.

![](../img/relationships.png)

There are some iterator functions available which can be implemented on iterable. You can check out these 2 links from GeekForGeeks. Pretty well written.

* [Iterator Functions | Set 1](https://www.geeksforgeeks.org/iterator-functions-in-python-set-1/)
* [Iterator Functions | Set 2](https://www.geeksforgeeks.org/iterator-functions-python-set-2islice-starmap-tee/)